<a href="https://colab.research.google.com/github/FRC4188/Machine-Learning/blob/main/tf_obejct_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Install Prerequistes


In [ ]:
!git clone https://github.com/nicknochnack/TFODCourse



Cloning into 'TFODCourse'...
remote: Enumerating objects: 89, done.
remote: Total 89 (delta 0), reused 0 (delta 0), pack-reused 89
Unpacking objects: 100% (89/89), done.


In [ ]:
!pip uninstall opencv-python -y
!pip install opencv-python
!pip install opencv-python-headless
!pip install tensorflow
!pip install tflite-runtime
!pip install roboflow
!pip install wget

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 60.4 MB 35 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 47.7 MB 111 kB/s 
     |████████████████████████████████| 462 kB 28.4 MB/s 
     |████████████████████████████████| 2.2 MB 25.2 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 145 kB 58.5 MB/s 
     |████████████████████████████████| 178 kB 35.2 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 138 kB 41.7 MB/s 
     |█

In [ ]:
import os
from roboflow import Roboflow
import wget
import tensorflow as tf

from google.protobuf import text_format
import os
import tensorflow as tf

import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Collect and Annotate Dataset

1. Take various pictures of the object from different angles, lighting, and distances. The more variations that you give the algorithm, the better it will be able to detect it in various circumstances.

2. Upload the pictures to a computer. If you're taking pictures with an iPhone, you will most likely need to convert its .heic format to .jpg. If this is the case, simply google "heic to jpg converter" and various programs will pop up for you. However, most of the online sites only allow you to convert up to 5 images at a time. You can download an app from one of the websites that will allow you to conver up to 20 images at a time. If you want to still use an iPhone but not go through the hassle, navigate to Settings>Camera>Formats and choose "Most Compatible". This will automatically take pictures in .jpeg form. 

3. Navigate to https://app.roboflow.com/ and create an account. This website is where we will label our images. Follow all the necessary steps to create an account. Once you have that, create a new project. Name the project. Name the annotation group a generic name for your object. For example, if I wanted to detect a Toyota, a Nissan, and a Ford vehicle, I would name the annotation group "car brands". Once you're inside your project, upload all the pictures you've taken. When they're done, click finish uploading. Then, there will be a pop-up that has a bar with the words "train", "valid", and "test". The numbers represents how the dataset will be split when it goes through the model. The model will train with the "train" picture and validate itself with the "valid" pictures. The "test" pictures will be shown to you at the end. In general, just go with the default splitting. 

4. Begin annotating. Click on the first image and draw a box around the object of interest. There will be a prompt at the top that will ask you to identify the object you just put a box around. For each different, object give it a different name. Once you've given it a name, click "save" or enter on your keyboard. Do this same process for all images in your dataset. If the object you're detection is generic enough, you may be able to use "label assist" on the sidebar. It will employ a mini-algorithm that will help you detect things in your images. 

5. Export your model version. Navigate out of the images by clicking the back arrow and then click "Dataset" on the sidebar. Generate a new version. You can add preprocessing steps in order to improve your training. Once you've chosen the desired steps, you can also choose augmentation steps. This will also assist in your training. Go to the next step and generate the model. When the version has finished generating, click on the version you just created and click "Export". On format, select "TFRecords". Also, select "show download code". Make sure "Jupyter" is selected. Copy each thing inside the quotations and in the code below, replace each the text with the appropriate data. 

#Download Dataset

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
rf = Roboflow(api_key="your-api-key")
project = rf.workspace("your-user").project("your-project-name")
dataset = project.version(your-version-number).download("your model")

%cd /content/datasets/ball-detection-2/test/ #change to match your model
os.rename('balls.tfrecord', 'test.record')
os.rename('balls_label_map.pbtxt', 'label_map.pbtxt')

%cd ../train
os.rename('balls.tfrecord', 'train.record')

%cd ../../../TFODCourse

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/ball-detection-2 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 450.31it/s]

/content/datasets/ball-detection-2/test
/content/datasets/ball-detection-2/train
/content/TFODCourse


# Setup Folders

In [ ]:
CUSTOM_MODEL_NAME = 'ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

In [ ]:
paths = {
    'TENSORFLOW_PATH': 'Tensorflow',
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG': os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt')
    }

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
            os.mkdir(path)

# Install Object Detection

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']} #ignore red squiggly under !git, no issue

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 69193, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 69193 (delta 10), reused 18 (delta 10), pack-reused 69174
Receiving objects: 100% (69193/69193), 577.29 MiB | 24.44 MiB/s, done.
Resolving deltas: 100% (48743/48743), done.


In [ ]:
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda

In [ ]:
import object_detection
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

#Verify Setup

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')

# Verify Installation; look for "OK" once cell completes
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-02-22 16:02:50.213347: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0222 16:02:50.967704 140570521892736 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 5.45s
I0222 16:02:52.130204 140570521892736 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 5.45s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 1.76s
I0222 16:02:53.891320 140570521892736 test

#Download Pretrained Model

In [ ]:
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-02-22 16:03:35--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.13.80, 2607:f8b0:4004:836::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.13.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  --.-KB/s    in 0.06s   

2022-02-22 16:03:35 (303 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8

#Move and Config Files

In [ ]:
!cp /content/datasets/ball-detection-2/test/test.record /content/TFODCourse/Tensorflow/workspace/annotations
!cp /content/datasets/ball-detection-2/test/label_map.pbtxt /content/TFODCourse/Tensorflow/workspace/annotations
!cp /content/datasets/ball-detection-2/train/train.record /content/TFODCourse/Tensorflow/workspace/annotations

In [ ]:
labelsfile = open('/content/TFODCourse/Tensorflow/workspace/annotations/label_map.pbtxt', "r")
filetxt = labelsfile.readlines()

count = 0
for line in filetxt:
  if line.find('id') != -1:
    count += 1
labelsfile.close()
print(count)

2


In [ ]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

pipeline_config.model.ssd.num_classes = count
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

#Train

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=800".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/ssd_mobnet/pipeline.config --num_train_steps=800


In [ ]:
!{command}

2022-02-22 16:10:26.558757: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0222 16:10:26.565583 140046593640320 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 800
I0222 16:10:26.570583 140046593640320 config_util.py:552] Maybe overwriting train_steps: 800
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0222 16:10:26.570851 140046593640320 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0222 16:10:26.755270 140046593640320 deprecation.py:343] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experimental

#Evaluate

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/ssd_mobnet/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/ssd_mobnet


In [ ]:
!{command}

W0222 16:45:29.727989 140274802063232 model_lib_v2.py:1090] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0222 16:45:29.728370 140274802063232 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0222 16:45:29.728495 140274802063232 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0222 16:45:29.728614 140274802063232 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0222 16:45:29.728758 140274802063232 model_lib_v2.py:1111] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2022-02-22 16:45:30.485163: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original confi

#View Training 

In [ ]:
%cd Tensorflow/workspace/models/train
%tensorboard --logdir=.
%cd ../../../../

[Errno 2] No such file or directory: 'Tensorflow/workspace/models/train'
/content/TFODCourse


UsageError: Line magic function `%tensorboard` not found.


#Load Model from Checkpoint

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-1')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

#Detect from Image

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'IMG_4891.jpg')

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=20,
            min_score_thresh=.9,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

ValueError: ignored

#Export Model as TFLite

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])
print(command)

python Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow/workspace/models/ssd_mobnet/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/ssd_mobnet --output_directory=Tensorflow/workspace/models/ssd_mobnet/export


In [ ]:
!{command}

2022-02-22 15:54:04.450310: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0222 15:54:04.638746 139782409283456 deprecation.py:615] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
2022-02-22 15:54:24.685560: W tensorflow/python/util/util.cc:368] Sets are not currently 

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])
print(command)

python Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py  --pipeline_config_path=Tensorflow/workspace/models/ssd_mobnet/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/ssd_mobnet --output_directory=Tensorflow/workspace/models/ssd_mobnet/tfliteexport


In [ ]:
!{command}

2022-02-22 15:55:04.736092: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-22 15:55:16.501017: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
W0222 15:55:19.161482 139920007571328 save_impl.py:72] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7f4118742190>, because it is not built.
W0222 15:55:19.494792 139920007571328 save_impl.py:72] Skipping full serialization of Keras layer <keras.layers.convolutional.SeparableConv2D object at 0x7f4118666590>, because it is not built.
W0222 15:55:19.495123 139920007571328 save_impl.py:72] Skipping full serialization of Keras layer <object_detection.core.freezable_batch_norm.FreezableBatchNorm object at 0x7f4117f25350>, because it is not built.
W0222 15:55:19.495262 139920007571328 s

In [ ]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )
print(command)

tflite_convert --saved_model_dir=Tensorflow/workspace/models/ssd_mobnet/tfliteexport/saved_model --output_file=Tensorflow/workspace/models/ssd_mobnet/tfliteexport/saved_model/detect.tflite --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [ ]:
!{command}

2022-02-22 15:55:54.317161: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-22 15:56:06.953124: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-02-22 15:56:06.953208: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
Estimated count of arithmetic ops: 1.702 G  ops, equivalently 0.851 G  MACs
W0222 15:56:11.182047 140555757066112 lite.py:770] Buffer deduplication procedure will be skipped when flatbuffer library is not properly loaded
